In [1]:
import os
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
from datetime import datetime
from models.handler import train, test
import argparse
import pandas as pd
from param_parser import parameter_parser, tab_printer

In [2]:
args = parameter_parser()

In [3]:
tab_printer(args)

+------------------------+----------+
|       Parameter        |  Value   |
+========================+==========+
| Batch size             | 32       |
+------------------------+----------+
| Dataset                | ECG_data |
+------------------------+----------+
| Decay rate             | 0.500    |
+------------------------+----------+
| Device                 | cpu      |
+------------------------+----------+
| Dropout rate           | 0.500    |
+------------------------+----------+
| Early stop             | 0        |
+------------------------+----------+
| Epoch                  | 2        |
+------------------------+----------+
| Evaluate               | 1        |
+------------------------+----------+
| Exponential decay step | 5        |
+------------------------+----------+
| Horizon                | 3        |
+------------------------+----------+
| Leakyrelu rate         | 0.200    |
+------------------------+----------+
| Lr                     | 0.000    |
+-----------

In [4]:
print(f'Training configs: {args}')

Training configs: Namespace(train=True, evaluate=True, dataset='ECG_data', window_size=12, horizon=3, train_length=7, valid_length=2, test_length=1, epoch=2, lr=0.0001, multi_layer=5, device='cpu', validate_freq=1, batch_size=32, norm_method='z_score', optimizer='RMSProp', early_stop=False, exponential_decay_step=5, decay_rate=0.5, dropout_rate=0.5, leakyrelu_rate=0.2)


In [5]:
data_file = os.path.join('dataset', args.dataset + '.csv')
result_train_file = os.path.join('output', args.dataset, 'train')
result_test_file = os.path.join('output', args.dataset, 'test')

In [6]:
data_file

'dataset\\ECG_data.csv'

In [7]:
result_train_file

'output\\ECG_data\\train'

In [8]:
result_test_file

'output\\ECG_data\\test'

In [9]:
# 如果文件夹不存在则创建文件夹
if not os.path.exists(result_train_file):
    os.makedirs(result_train_file)
if not os.path.exists(result_test_file):
    os.makedirs(result_test_file)

In [10]:
# 导入数据文件
data = pd.read_csv(data_file).values

In [11]:
data.shape

(4999, 140)

### split data

In [12]:
train_ratio = args.train_length / (args.train_length + args.valid_length + args.test_length)
valid_ratio = args.valid_length / (args.train_length + args.valid_length + args.test_length)
test_ratio = 1 - train_ratio - valid_ratio

In [13]:
print(train_ratio, valid_ratio, test_ratio)

0.7 0.2 0.10000000000000003


In [14]:
train_data = data[:int(train_ratio * len(data))]
valid_data = data[int(train_ratio * len(data)):int((train_ratio + valid_ratio) * len(data))]
test_data = data[int((train_ratio + valid_ratio) * len(data)):]

In [15]:
print(train_data.shape, valid_data.shape, test_data.shape)

(3499, 140) (1000, 140) (500, 140)


In [16]:
torch.manual_seed(0)

### 模型训练

In [17]:
before_train = datetime.now().timestamp()
before_train

1700664675.547299

In [18]:
_, normalize_statistic = train(train_data, valid_data, args, result_train_file)

Total Trainable Params: 1123303


| end of epoch   0 | time: 48.61s | train_total_loss 0.8796
------ validate on data: VALIDATE ------
NORM: MAPE 94.731540050%; MAE 0.737967421; RMSE 0.987366123.
RAW : MAPE 95.622746304%; MAE 0.404625008; RMSE 0.692389483.
| end of epoch   1 | time: 45.04s | train_total_loss 0.8288
------ validate on data: VALIDATE ------
NORM: MAPE 95.037781538%; MAE 0.687440681; RMSE 0.943461815.
RAW : MAPE 92.255530838%; MAE 0.369621268; RMSE 0.638351044.


In [19]:
after_train = datetime.now().timestamp()

In [20]:
print(f'Training took {(after_train - before_train) / 60} minutes')

Training took 1.7450101335843404 minutes


In [21]:
before_evaluation = datetime.now().timestamp()

In [22]:
test(test_data, args, result_train_file, result_test_file)

NORM: MAPE 111.701000052%; MAE 0.648621949; RMSE 0.917062778.
RAW : MAPE 88.679524567%; MAE 0.330217093; RMSE 0.551628180.
Performance on test set: MAPE:  0.89 | MAE:  0.33 | RMSE: 0.5516


In [23]:
after_evaluation = datetime.now().timestamp()
print(f'Evaluation took {(after_evaluation - before_evaluation) / 60} minutes')

Evaluation took 0.04551933606465657 minutes
